In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 134.9 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# common imports

import sys
assert sys.version_info >= (3, 5)


import sklearn
assert sklearn.__version__ >= "0.20"


import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os

np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [2]:
import datasets

dataset = datasets.load_from_disk('../../datasets/ARID_supporting_scripts/5_1_training_set')
dataset = datasets.concatenate_datasets([dataset['train'], dataset['test']])

In [3]:
dataset.features['label'].names

['may_signal_keyword_general_text',
 'may_signal_keyword_requirement',
 'may_signal_keyword_srs_text',
 'must_signal_keyword_general_text',
 'must_signal_keyword_requirement',
 'must_signal_keyword_srs_text',
 'no_signal_keyword_general_text',
 'no_signal_keyword_srs_text',
 'shall_signal_keyword_general_text',
 'shall_signal_keyword_requirement',
 'shall_signal_keyword_srs_text',
 'should_signal_keyword_general_text',
 'should_signal_keyword_requirement',
 'should_signal_keyword_srs_text',
 'will_signal_keyword_general_text',
 'will_signal_keyword_requirement',
 'will_signal_keyword_srs_text']

In [4]:
import evaluate
from transformers.keras_callbacks import KerasMetricCallback

def compute_metrics(eval_predictions):
    metric1 = evaluate.load("precision")
    metric2 = evaluate.load("recall")
    metric3 = evaluate.load("f1")

    predictions, labels = eval_predictions
    predictions = np.argmax(predictions, axis = 1)

    precision = metric1.compute(predictions = predictions, references = labels, average = 'macro')["precision"]
    recall = metric2.compute(predictions = predictions, references = labels, average = 'macro')["recall"]
    f1_weighted = metric3.compute(predictions = predictions, references = labels, average = 'macro')["f1"]
    return {"precision": precision, "recall": recall, 'f1_macro': f1_weighted}

2026-01-13 00:00:11.350225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-13 00:00:11.350265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-13 00:00:11.351697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-13 00:00:11.357930: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-13 00:00:11.910719: W tensorflow/compiler/tf2

In [5]:
from transformers import create_optimizer
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


def train_fold(model_ckpt, encoded_tt_splits, lbl_, save_path):
    label2id = {lbl: idx for idx, lbl in enumerate(lbl_)}
    id2label = {val: key for key, val in label2id.items()}

    model = TFAutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = len(lbl_), id2label = id2label, label2id = label2id,)
    
    tf_train_dataset = model.prepare_tf_dataset(encoded_tt_splits['train'], shuffle = True, batch_size = batch_size, tokenizer = tokenizer)
    tf_valid_dataset = model.prepare_tf_dataset(encoded_tt_splits['test'], shuffle = False, batch_size = batch_size, tokenizer = tokenizer)

    batches_per_epoch = len(encoded_tt_splits['train']) // batch_size
    total_train_steps = int(batches_per_epoch * num_epochs)
    optimizer, schedule = create_optimizer(init_lr = 2e-5, num_warmup_steps = 0, num_train_steps = total_train_steps)
    
    metric_callback = KerasMetricCallback(metric_fn = compute_metrics, eval_dataset = tf_valid_dataset)
    
    model.compile(optimizer = optimizer)
    history = model.fit(tf_train_dataset, validation_data = (tf_valid_dataset), epochs = num_epochs, callbacks = [metric_callback])

    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

In [6]:
def preprocess_function(dataset):
    return tokenizer(dataset['Requirement Sentences'], truncation = True)

In [7]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

K = 10
batch_size = 16
num_epochs = 30
model_ckpt = 'google-bert/bert-base-cased'
model_name = model_ckpt.split('/')[-1]
label_names = dataset.features['label'].names

labels = np.array(dataset["label"])
skf = StratifiedKFold(n_splits = K, shuffle = True, random_state = 42)

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

for fold, (train_idx, test_idx) in enumerate(skf.split(np.zeros(len(labels)), labels), start = 1):
    print(f'[TRAINING] Fold {fold}/{K}')
    train_split = dataset.select(train_idx.tolist())
    test_split  = dataset.select(test_idx.tolist())
    train_data = datasets.DatasetDict({"train": train_split, "test": test_split})
    encoded_tt_splits = train_data.map(preprocess_function, batched = True)        
    
    save_path = f'./models/tuned_10_fold/reqseek_bert_base_cased_kfold_trained/trained_fold_{fold}'
    train_fold(model_ckpt, encoded_tt_splits, label_names, save_path)

    train_data.save_to_disk(f'./models_10fold_dataset_splits/kfold_bert_base_cased_data/train_test_fold_{fold}')

[TRAINING] Fold 1/10


Map:   0%|          | 0/2156 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

2026-01-13 00:00:13.661130: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-01-13 00:00:13.662773: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2026-01-13 00:00:13.665308: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/30
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

134/134 [==============================] - 23s 175ms/step - loss: 1.2361 - val_loss: 1.0048 - precision: 0.6377 - recall: 0.5690 - f1_macro: 0.5433
Epoch 3/30


/home/kasra/metal-engine/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


134/134 [==============================] - 23s 169ms/step - loss: 0.8241 - val_loss: 0.6935 - precision: 0.8009 - recall: 0.7476 - f1_macro: 0.7529
Epoch 4/30
134/134 [==============================] - 22s 159ms/step - loss: 0.5914 - val_loss: 0.5655 - precision: 0.8604 - recall: 0.8253 - f1_macro: 0.8298
Epoch 5/30
134/134 [==============================] - 22s 161ms/step - loss: 0.4362 - val_loss: 0.5356 - precision: 0.8118 - recall: 0.7821 - f1_macro: 0.7830
Epoch 6/30
134/134 [==============================] - 21s 157ms/step - loss: 0.3435 - val_loss: 0.5260 - precision: 0.8103 - recall: 0.7955 - f1_macro: 0.7888
Epoch 7/30
134/134 [==============================] - 21s 156ms/step - loss: 0.2587 - val_loss: 0.4702 - precision: 0.8300 - recall: 0.8236 - f1_macro: 0.8195
Epoch 8/30
134/134 [==============================] - 20s 149ms/step - loss: 0.1927 - val_loss: 0.5330 - precision: 0.8124 - recall: 0.7999 - f1_macro: 0.7982
Epoch 9/30
134/134 [==============================] - 20s

Saving the dataset (0/1 shards):   0%|          | 0/2157 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/239 [00:00<?, ? examples/s]